# gazeNet 

## Data upload
Copy your folder into 'etdata' with .tsv files to the left panel.

In [ ]:
import glob

root = 'etdata'         # The root folder of your trials
dataset = 'trials'      # Your folder name
format = 'tsv'

FILES = glob.glob('%s/%s/*.%s' % (root, dataset, format))

## Input trial setup

Enter measurements for Your eye-tracking-setup.

The default values for monitor width and height are for a 24" 16:9 monitor.

In [ ]:
geom = {
    'screen_width': 53.1,       # Screen width in cm
    'screen_height': 29.8,      # Screen height in cm
    'display_width_pix': 1920,  # Screen width in pixels
    'display_height_pix': 1080, # Screen height in pixels
    'eye_distance': 60,         # Viewing distance in cm
}

In [ ]:
import os
import numpy as np
import pandas as pd

## model configuration
from utils_lib import utils

dev = False
model_dir = 'model_final'
model_name = 'gazeNET_0004_00003750'
model_name = '%s.pth.tar'%model_name

logdir =  os.path.join('logdir', model_dir)
fname_config = os.path.join(logdir, 'config.json')
configuration = utils.Config(fname_config)
config = configuration.params

config['split_seqs']=False
config['augment']=False
config['batch_size']=1
cuda = False
num_classes = len(config['events'])

## load model
from model import gazeNET as gazeNET
import model as model_func

model = gazeNET(config, num_classes)
model_func.load(model, model_dir, config, model_name)
model.eval()


In [5]:

## load data
from utils_lib.etdata import ETData, tsv_to_npy
from utils_lib.data_loader import EMDataset, GazeDataLoader

for fpath in FILES:
    fname = os.path.basename(fpath)
    if format == 'tsv':
        x_px, y_px, X_test = tsv_to_npy(fpath, geom)
    else:
        X_test = np.load(fpath)

    _status = np.isnan(X_test['x']) | \
            np.isnan(X_test['y']) | \
            ~np.in1d(X_test['evt'], config['events'])
    X_test['status'] = ~_status
    test_dataset = EMDataset(config = config, gaze_data = [X_test])
    n_samples = len(test_dataset)
    test_loader = GazeDataLoader(test_dataset, batch_size=config['batch_size'],
                                num_workers=1,
                                shuffle=False)
    
    ## label data
    from utils_lib.ETeval import run_infer
    kwargs = {
        'cuda': False,
        'use_tqdm': False,
        'eval': False,
    }

    print ("Predicting %s" % fname)
    _gt, _pr, pr_raw = run_infer(model, n_samples, test_loader, **kwargs)

    ## postprocessing

    # revert to kartesian
    if format == 'tsv':
        X_test['x'] = x_px
        X_test['y'] = y_px

    #glue back the predictions
    import copy

    _data_pr = copy.deepcopy(test_dataset.data)
    for _d, _pred in zip(_data_pr, pr_raw):
        _d['evt'] = 0
        _d['evt'][1:] = np.argmax(_pred, axis=1)+1
    _data_pr = pd.concat([pd.DataFrame(_d) for _d in _data_pr]).reset_index(drop=True)
    _data = pd.DataFrame(X_test)
    _data = _data.merge(_data_pr, on='t', suffixes=('', '_pred'), how='left')
    _data['evt'] = _data['evt_pred'].replace({np.nan:0})

    etdata_pr = ETData()
    etdata_pr.load(_data[['t', 'x', 'y', 'status', 'evt']].values, **{'source':'np_array'})

    sdir = '%s/%s_gazeNet'%(root, dataset)
    if not os.path.exists(sdir):
        os.makedirs(sdir)
    spath = '%s/%s'%(sdir, fname.replace('.tsv', ''))
    etdata_pr.save(spath)
    etdata_pr.plot_px(show=False, save=True, spath='%s'%spath)
    etdata_pr.plot_xy(show=False, save=True, spath='%s'%spath)

: 

## Download files

In [ ]:
import solara

f = open(fpath, "rb")
solara.FileDownload(data=f, label="Download image", filename="image.jpeg")
widgets.VBox([
    solara.FileDownload.widget(data=f, label="Download image", filename=fpath)
])
display(solara.FileDownload.widget(data=f, label="Download image", filename=fpath))
